In [1]:
#IMPORTANT NOTE: we have not used the suits of the cards in this codes to break ties.

#FILL THIS IN WITH THE DESIRED/APPROPRIATE PARAMETERS BEFORE USING THIS CODE.
dataset_version = '50k'
num_rows = 50000

In [2]:
import random
import statistics as st
import pandas as pd
from itertools import combinations
import time
random.seed(0)

In [3]:
def four_player_ds(number_of_rows):
    trials = []
    for x in range(number_of_rows):
        deck = [(1,1),(1,2),(1,3),(1,4),(1,5),(1,6),(1,7),(1,8),(1,9),(1,10),(1,11),(1,12),(1,13),(2,1),(2,2),(2,3),(2,4),(2,5),(2,6),(2,7),(2,8),(2,9),(2,10),(2,11),(2,12),(2,13),(3,1),(3,2),(3,3),(3,4),(3,5),(3,6),(3,7),(3,8),(3,9),(3,10),(3,11),(3,12),(3,13),(4,1),(4,2),(4,3),(4,4),(4,5),(4,6),(4,7),(4,8),(4,9),(4,10),(4,11),(4,12),(4,13)]
        p1 = []
        p2 = []
        p3 = []
        p4 = []
        i = 1
        while i < 14:
            p1.append(deck.pop(random.randint(0,52-i)))
            i += 1
        while i < 27:
            p2.append(deck.pop(random.randint(0,52-i)))
            i += 1
        while i < 40:
            p3.append(deck.pop(random.randint(0,52-i)))
            i += 1
        while i < 53:
            p4.append(deck.pop(random.randint(0,52-i)))
            i += 1
        trial = (p1,p2,p3,p4)
        trials.append(trial)

    p1 = []
    p2 = []
    p3 = []
    p4 = []
    for x in trials:
        p1 += [x[0]]
        p2 += [x[1]]
        p3 += [x[2]]
        p4 += [x[3]]

    trials = pd.DataFrame({'p1':p1, 'p2':p2, 'p3':p3, 'p4':p4})
    return trials

def combinations_converted(hand,length):
    '''
    converted the combinations from itertools function to a more useful form, in this context.
    
    input:    [((1, 2), (1, 3)), ((1, 2), (4, 4)), ((1, 3), (4, 4))]
    output:   [[(1, 2), (1, 3)], [(1, 2), (4, 4)], [(1, 3), (4, 4)]]
    it changes the tuples in the first list to lists.
    '''
    comb = combinations(hand,length)
    combi = []
    for x in comb:
        combi += [x]
    combi_convert = []
    for y in range(len(combi)):
        combi_convert += [[]]
    for y in range(len(combi)):
        for z in range(len(combi[0])):
            combi_convert[y] += [combi[y][z]]
    return combi_convert

def equal_hands(hand_1,hand_2):
    '''
    input: two hands, that are lists, that contain cards with number 0-52 each
    output: if the two hands are equal : True
            else: False
    '''
    hand_1copy = hand_1.copy()
    hand_2copy = hand_2.copy()
    hand_1copy.sort()
    hand_2copy.sort()
    
    if len(hand_1copy) == len(hand_2copy):
        counter = 0
        for x in range(len(hand_1copy)):
            if hand_1copy[x][0] == hand_2copy[x][0] and hand_1copy[x][1] == hand_2copy[x][1]:
                counter += 1
        if counter == len(hand_1copy):
            return True
        else:
            return False
    else:
        return False

def check_duplicates_inasetofhands(set):
    '''
    input: a set,that is a list, that contains hand/s with number of cards from 0-52
    output: if duplication exist: (True, indices of first duplication)
            else: False
    '''
    if len(set) == 1 or len(set) == 0:
        return False
    
    orig_index_list = []
    orig_less_last = []
    for x in range(len(set)):
        orig_index_list += [x]
        orig_less_last += [x]
    orig_less_last.remove(orig_less_last[-1])

    for x in orig_less_last:
        for y in orig_index_list[x+1:len(orig_index_list)]:
            if equal_hands(set[x],set[y]) == True:
                return (True, (x,y))
    return False

def intersection_setsofhands(set_1,set_2):
    '''
    input: two sets,that are lists, that contain hands with number of cards from 0-52 and that each of the two sets
    must have NO DUPLICATION. 
    output: a list that contains the common elements of the two sets
    '''
    intersection = []
    for hand_1 in set_1:
        for hand_2 in set_2:
            if equal_hands(hand_1,hand_2) == True:
                intersection += [hand_1]
                break
    return intersection

def intersection_hands(h1,h2):
    intersection = []
    for card1 in h1:
        for card2 in h2:
            if card1 == card2:
                intersection += [card1]
                break
    return intersection

def straight_5(hand,rank,rank_no_dupli,count):
    '''
    input: hand is a list that contains 13 or 8 or 10 or 5 cards drawn from a standard deck without replacement
    with format (suit(4-diamond),rank(13-ace)).
        rank is a list that contains the rank of each card of a hand. it can have duplicated values.
        rank_no_dupli is a list that contains the ranks that are present in the hand. 
    it has no duplciated values and must be of descending order.
        count is a dictionary that contains the count of each rank in the hand.        
        
    output: (a,b)
    a := len(b)
    b := a list that contains all the straight/s of length 5 that can be created from the hand. the list is sorted in descending order.
    '''
    #early exit to save time
    if len(rank_no_dupli) < 5:
        return (0,[])
    
    types_straights = [] #ex. high of 5
    consecutives = []
    for x in range(len(rank_no_dupli)-2):
        candidate = [rank_no_dupli[x]]
        for y in range(x + 1, len(rank_no_dupli)):
            if candidate[-1] - 1 == rank_no_dupli[y]:
                candidate += [rank_no_dupli[y]]
                if len(candidate) == 5:
                    break
            else:
                break
        consecutives += [candidate]
    for x in consecutives:
        if len(x) == 5:
            types_straights += [x]

    a = 0
    for x in [4,3,2,1,13]:
        if count[x] >= 1:
            a += 1
    if a == 5:
        types_straights += [[4,3,2,1,13]]

    if len(types_straights) == 0:
        return (0,[])

    #straights construction
    straights = []
    
    for x in types_straights:
        #p here is the number of ways one can create the type of straight of interest, which is x.
        p = count[x[0]] * count[x[1]] * count[x[2]] * count[x[3]] * count[x[4]]
        straights_type_x = []
        for y in range(p):
            straights_type_x += [[]]

        #card setting
        partition_size = p
        partition = 1
        for y in x:
            cards_with_rank_y = []
            for card in hand:
                if card[1] == y:
                    cards_with_rank_y += [card]
            slots_per_card = partition_size/len(cards_with_rank_y)
            c = 0
            for z in range(partition):
                for card in cards_with_rank_y:
                    i = 0
                    while i < slots_per_card:
                        straights_type_x[c] += [card]
                        i += 1
                        c += 1
            partition_size = int(partition_size / len(cards_with_rank_y))
            partition = partition * len(cards_with_rank_y)
        straights.extend(straights_type_x)    

    return (len(straights),straights)

def straight_3(hand,rank,rank_no_dupli,count):
    '''
    input: hand is a list that contains 13 or 8 or 3 cards drawn from a standard deck without replacement
    with format (suit(4-diamond),rank(13-ace)).
        rank is a list that contains the rank of each card of a hand. it can have duplicated values.
        rank_no_dupli is a list that contains the ranks that are present in the hand. 
    it has no duplciated values and must be of descending order.
        count is a dictionary that contains the count of each rank in the hand.        
        
    output: (a,b)
    a := len(b)
    b := a list that contains all the straight/s of length 3 that can be created from the hand. the list is sorted in descending order.
    '''
    if len(rank_no_dupli) < 3:
        return (0,[])
    
    types_straights = [] #ex. high of 5
    consecutives = []
    for x in range(len(rank_no_dupli)-2):
        candidate = [rank_no_dupli[x]]
        for y in range(x + 1, len(rank_no_dupli)):
            if candidate[-1] - 1 == rank_no_dupli[y]:
                candidate += [rank_no_dupli[y]]
                if len(candidate) == 3:
                    break
            else:
                break
        consecutives += [candidate]
    for x in consecutives:
        if len(x) == 3:
            types_straights += [x]

    a = 0
    for x in [2,1,13]:
        a += count[x]
    if a >= 3:
        types_straights += [[2,1,13]]

    if len(types_straights) == 0:
        return (0,[])

    #straights construction
    straights = []
    
    for x in types_straights:
        #p here is the number of ways one can create the type of straight of interest, which is x.
        p = count[x[0]] * count[x[1]] * count[x[2]]
        straights_type_x = []
        for y in range(p):
            straights_type_x += [[]]

        #card setting
        partition_size = p
        partition = 1
        for y in x:
            cards_with_rank_y = []
            for card in hand:
                if card[1] == y:
                    cards_with_rank_y += [card]
            slots_per_card = partition_size/len(cards_with_rank_y)
            c = 0
            for z in range(partition):
                for card in cards_with_rank_y:
                    i = 0
                    while i < slots_per_card:
                        straights_type_x[c] += [card]
                        i += 1
                        c += 1
            partition_size = int(partition_size / len(cards_with_rank_y))
            partition = partition * len(cards_with_rank_y)
        straights.extend(straights_type_x)    

    return (len(straights),straights)

def royal_flush_5(hand,rank,rank_no_dupli,count):
    '''
    input: hand is a list that contains 13 or 8 or 10 or 5 cards drawn from a standard deck without replacement
    with format (suit(4-diamond),rank(13-ace)).
        rank is a list that contains the rank of each card of a hand. it can have duplicated values.
        rank_no_dupli is a list that contains the ranks that are present in the hand. 
    it has no duplciated values and must be of descending order.
        count is a dictionary that contains the count of each rank in the hand.
        
    output: (a,b)
    a := len(b)
    b := a list that contains all the royal flush/s of length 5 that can be created from the hand.
    '''
    royalflushes = []
    for x in [[(1,13),(1,12),(1,11),(1,10),(1,9)],[(2,13),(2,12),(2,11),(2,10),(2,9)],[(3,13),(3,12),(3,11),(3,10),(3,9)],[(4,13),(4,12),(4,11),(4,10),(4,9)]]:
        temp = intersection_hands(hand,x)
        if len(temp) == 5:
            #sort temp first before giving it to royalflushes
            suit = temp[0][0]
            temp = [(suit,13),(suit,12),(suit,11),(suit,10),(suit,9)]
            royalflushes += [temp]
            
    return (len(royalflushes),royalflushes)

def flush_5(hand,rank,rank_no_dupli,count):
    '''
    input: hand is a list that contains 13 or 8 or 10 or 5 cards drawn from a standard deck without replacement
    with format (suit(4-diamond),rank(13-ace)).
        rank is a list that contains the rank of each card of a hand. it can have duplicated values.
        rank_no_dupli is a list that contains the ranks that are present in the hand. 
    it has no duplciated values and must be of descending order.
        count is a dictionary that contains the count of each rank in the hand.
        
    output: (a,b)
    a := len(b)
    b := a list that contains all the flush/s of length 5 that can be created from the hand. the list is sorted in descending order.
    '''
    suit = []
    for card in hand:
        suit += [card[0]]
    
    flushes = []
    for x in range(1,5):
        flush_cards = []
        if suit.count(x) >= 5:
            for card in hand:
                if card[0] == x:
                    flush_cards.append(card)
            combi_convert = combinations_converted(flush_cards,5)
            for y in combi_convert:
                y.sort(reverse=True)
            combi_convert.sort(reverse=True)
            for x in combi_convert:
                flushes += [x]

    if len(flushes) == 0:
        return (0,[])
    else:
        return (len(flushes),flushes)
    
def flush_3(hand,rank,rank_no_dupli,count):
    '''
    input: hand is a list that contains 13 or 8 or 3 cards drawn from a standard deck without replacement
    with format (suit(4-diamond),rank(13-ace)).
        rank is a list that contains the rank of each card of a hand. it can have duplicated values.
        rank_no_dupli is a list that contains the ranks that are present in the hand. 
    it has no duplciated values and must be of descending order.
        count is a dictionary that contains the count of each rank in the hand.
        
    output: (a,b)
    a := len(b)
    b := a list that contains all the flush/s of length 3 that can be created from the hand. the list is sorted in descending order.
    '''
    suit = []
    for card in hand:
        suit += [card[0]]
    
    flushes = []
    for x in range(1,5):
        flush_cards = []
        if suit.count(x) >= 3:
            for card in hand:
                if card[0] == x:
                    flush_cards.append(card)
            combi_convert = combinations_converted(flush_cards,3)
            for y in combi_convert:
                y.sort(reverse=True)
            combi_convert.sort(reverse=True)
            for x in combi_convert:
                flushes += [x]

    if len(flushes) == 0:
        return (0,[])
    else:
        return (len(flushes),flushes)
    
def straight_flush_5(hand,rank,rank_no_dupli,count):
    '''
    input: hand is a list that contains 13 or 8 or 10 or 5 cards drawn from a standard deck without replacement
    with format (suit(4-diamond),rank(13-ace)).
        rank is a list that contains the rank of each card of a hand. it can have duplicated values.
        rank_no_dupli is a list that contains the ranks that are present in the hand. 
    it has no duplciated values and must be of descending order.
        count is a dictionary that contains the count of each rank in the hand.
        
    output: (a,b)
    a := len(b)
    b := a list that contains all the straight flush/s of length 5 that can be created from the hand. the list is sorted in descending order.
    '''
    a = straight_5(hand,rank,rank_no_dupli,count)[1]
    b = flush_5(hand,rank,rank_no_dupli,count)[1]
    straightflushes = intersection_setsofhands(a,b)
    if len(straightflushes) == 0:
        return (0,[])
    else:
        for x in [[(1,13),(1,12),(1,11),(1,10),(1,9)],[(2,13),(2,12),(2,11),(2,10),(2,9)],[(3,13),(3,12),(3,11),(3,10),(3,9)],[(4,13),(4,12),(4,11),(4,10),(4,9)]]:
            if len(intersection_setsofhands([x],straightflushes)) != 0:
                straightflushes.remove(x)
    if len(straightflushes) == 0:
        return (0,[])
    else:
        return (len(straightflushes),straightflushes)
    
def quad_5(hand,rank,rank_no_dupli,count):
    '''
    input: hand is a list that contains 13 or 10 or 8 or 5 cards drawn from a standard deck without replacement
    with format (suit(4-diamond),rank(13-ace)).
        rank is a list that contains the rank of each card of a hand. it can have duplicated values.
        rank_no_dupli is a list that contains the ranks that are present in the hand. 
    it has no duplciated values and must be of descending order.
        count is a dictionary that contains the count of each rank in the hand.
        
    output: (a,b)
    a := len(b)
    b := a list that contains all the quad/s of length 5 that can be created from the hand. i think the order of b dont matter.
    '''
    quads = []
    qs = []
    for x in rank_no_dupli:
        if count[x] == 4:
            qs += [x]

    if len(qs) == 0:
        return (0,[])
            
    for x in qs:
        hand_less_q = []
        rank_no_dupli_less_q = rank_no_dupli.copy()
        rank_no_dupli_less_q.remove(x)
        for y in rank_no_dupli_less_q:
            for card in hand:
                if card[1] == y:
                    hand_less_q += [card]

        q_x = []
        for y in hand_less_q:
            q_x = [(1,x),(2,x),(3,x),(4,x),y]
            quads += [q_x]
    
    return (len(quads),quads)

def high_card_5(hand,rank,rank_no_dupli,count):
    '''
    input: hand is a list that contains at least 5 cards drawn from a standard deck without replacement
    with format (suit(4-diamond),rank(13-ace)).
        rank is a list that contains the rank of each card of a hand. it can have duplicated values.
        rank_no_dupli is a list that contains the ranks that are present in the hand. 
    it has no duplciated values and must be of descending order.
        count is a dictionary that contains the count of each rank in the hand.
        
    output: (a,b)
    a := len(b)
    b := a list that contains all high cards/s of length 5 that can be created from the hand. the list is sorted in descending order.
    '''
    high_cards = []
    
    combi = combinations_converted(rank_no_dupli,5)
    
    for x in combi:
        #construct an equivalent of p in the straight function
        p = 1
        for y in x:
            p *= count[y]
        hc_type_x = []
        for y in range(p):
            hc_type_x += [[]]
            
        #card setting
        partition_size = p
        partition = 1
        for y in x:
            cards_with_rank_y = []
            for card in hand:
                if card[1] == y:
                    cards_with_rank_y += [card]
            slots_per_card = partition_size/len(cards_with_rank_y)
            c = 0
            for z in range(partition):
                for card in cards_with_rank_y:
                    i = 0
                    while i < slots_per_card:
                        hc_type_x[c] += [card]
                        i += 1
                        c += 1
            partition_size = int(partition_size / len(cards_with_rank_y))
            partition = partition * len(cards_with_rank_y)
        high_cards.extend(hc_type_x)
    
    return (len(high_cards),high_cards)

def high_card_3(hand,rank,rank_no_dupli,count):
    '''
    input: hand is a list that contains at least 3 cards drawn from a standard deck without replacement
    with format (suit(4-diamond),rank(13-ace)).
        rank is a list that contains the rank of each card of a hand. it can have duplicated values.
        rank_no_dupli is a list that contains the ranks that are present in the hand. 
    it has no duplciated values and must be of descending order.
        count is a dictionary that contains the count of each rank in the hand.
        
    output: (a,b)
    a := len(b)
    b := a list that contains all high cards/s of length 3 that can be created from the hand. the list is sorted in descending order.
    '''
    high_cards = []
    
    combi = combinations_converted(rank_no_dupli,3)
    
    for x in combi:
        #construct an equivalent of p in the straight function
        p = 1
        for y in x:
            p *= count[y]
        hc_type_x = []
        for y in range(p):
            hc_type_x += [[]]
            
        #card setting
        partition_size = p
        partition = 1
        for y in x:
            cards_with_rank_y = []
            for card in hand:
                if card[1] == y:
                    cards_with_rank_y += [card]
            slots_per_card = partition_size/len(cards_with_rank_y)
            c = 0
            for z in range(partition):
                for card in cards_with_rank_y:
                    i = 0
                    while i < slots_per_card:
                        hc_type_x[c] += [card]
                        i += 1
                        c += 1
            partition_size = int(partition_size / len(cards_with_rank_y))
            partition = partition * len(cards_with_rank_y)
        high_cards.extend(hc_type_x)
    
    return (len(high_cards),high_cards)

def high_card_2(hand,rank,rank_no_dupli,count):
    '''
    input: hand is a list that contains at least 2 cards drawn from a standard deck without replacement
    with format (suit(4-diamond),rank(13-ace)).
        rank is a list that contains the rank of each card of a hand. it can have duplicated values.
        rank_no_dupli is a list that contains the ranks that are present in the hand. 
    it has no duplciated values and must be of descending order.
        count is a dictionary that contains the count of each rank in the hand.
        
    output: (a,b)
    a := len(b)
    b := a list that contains all high cards/s of length 2 that can be created from the hand. the list is sorted in descending order.
    '''
    high_cards = []
    
    combi = combinations_converted(rank_no_dupli,2)
    
    for x in combi:
        #construct an equivalent of p in the straight function
        p = 1
        for y in x:
            p *= count[y]
        hc_type_x = []
        for y in range(p):
            hc_type_x += [[]]
            
        #card setting
        partition_size = p
        partition = 1
        for y in x:
            cards_with_rank_y = []
            for card in hand:
                if card[1] == y:
                    cards_with_rank_y += [card]
            slots_per_card = partition_size/len(cards_with_rank_y)
            c = 0
            for z in range(partition):
                for card in cards_with_rank_y:
                    i = 0
                    while i < slots_per_card:
                        hc_type_x[c] += [card]
                        i += 1
                        c += 1
            partition_size = int(partition_size / len(cards_with_rank_y))
            partition = partition * len(cards_with_rank_y)
        high_cards.extend(hc_type_x)
    
    return (len(high_cards),high_cards)

def high_card_1(hand,rank,rank_no_dupli,count):
    '''
    input: hand is a list that contains at least 2 cards drawn from a standard deck without replacement
    with format (suit(4-diamond),rank(13-ace)).
        rank is a list that contains the rank of each card of a hand. it can have duplicated values.
        rank_no_dupli is a list that contains the ranks that are present in the hand. 
    it has no duplciated values and must be of descending order.
        count is a dictionary that contains the count of each rank in the hand.
        
    output: (a,b)
    a := len(b)
    b := a list that contains all high cards/s of length 2 that can be created from the hand. the list is sorted in descending order.
    '''
    high_cards = []
    
    combi = combinations_converted(rank_no_dupli,1)
    
    for x in combi:
        #construct an equivalent of p in the straight function
        p = 1
        for y in x:
            p *= count[y]
        hc_type_x = []
        for y in range(p):
            hc_type_x += [[]]
            
        #card setting
        partition_size = p
        partition = 1
        for y in x:
            cards_with_rank_y = []
            for card in hand:
                if card[1] == y:
                    cards_with_rank_y += [card]
            slots_per_card = partition_size/len(cards_with_rank_y)
            c = 0
            for z in range(partition):
                for card in cards_with_rank_y:
                    i = 0
                    while i < slots_per_card:
                        hc_type_x[c] += [card]
                        i += 1
                        c += 1
            partition_size = int(partition_size / len(cards_with_rank_y))
            partition = partition * len(cards_with_rank_y)
        high_cards.extend(hc_type_x)
    
    return (len(high_cards),high_cards)

def trio_5(hand,rank,rank_no_dupli,count):
    '''
    input: hand is a list that contains 13 or 10 or 8 or 5 cards drawn from a standard deck without replacement
    with format (suit(4-diamond),rank(13-ace)).
        rank is a list that contains the rank of each card of a hand. it can have duplicated values.
        rank_no_dupli is a list that contains the ranks that are present in the hand. 
    it has no duplciated values and must be of descending order.
        count is a dictionary that contains the count of each rank in the hand.
        
    output: (a,b)
    a := len(b)
    b := a list that contains all the trio/s of length 5 that can be created from the hand. the list is sorted in descending order.
    '''
    ts = []
    for x in rank_no_dupli:
        if count[x] >= 3:
            ts += [x]
    
    if len(ts) == 0:
        return (0,[])
    
    #trios construction, which is the final product
    trios = []
    
    for x in ts:
        #first case
        if count[x] == 3:
            hand_less_t = []
            for card in hand:
                if card[1] != x:
                    hand_less_t += [card]
            rank_hand_less_t = []
            for card in hand_less_t:
                rank_hand_less_t.append(card[1])
            rank_no_dupli_hand_less_t = list(set(rank_hand_less_t))
            rank_no_dupli_hand_less_t.sort(reverse=True)
            count_hand_less_t = {}
            for y in rank_no_dupli_hand_less_t:
                count_hand_less_t[y] = rank_hand_less_t.count(y)
            
            #gather up all the high cards, then permute them with the only trio x
            high_cards = high_card_2(hand_less_t,rank_hand_less_t,rank_no_dupli_hand_less_t,count_hand_less_t)[1]
            
            if len(high_cards) != 0:
                #this list is the trio, next is to append the high cards to it
                t_x = []
                for card in hand:
                    if card[1] == x:
                        t_x += [card]
                for y in high_cards:
                    t_x_with_h_c = t_x.copy()
                    t_x_with_h_c.extend(y)
                    trios += [t_x_with_h_c]
                    
        #second case
        elif count[x] == 4:
            hand_less_t = []
            for card in hand:
                if card[1] != x:
                    hand_less_t += [card]
            rank_hand_less_t = []
            for card in hand_less_t:
                rank_hand_less_t.append(card[1])
            rank_no_dupli_hand_less_t = list(set(rank_hand_less_t))
            rank_no_dupli_hand_less_t.sort(reverse=True)
            count_hand_less_t = {}
            for y in rank_no_dupli_hand_less_t:
                count_hand_less_t[y] = rank_hand_less_t.count(y)
                    
            #gather up all the high cards, then permute them with the only trio x
            high_cards = high_card_2(hand_less_t,rank_hand_less_t,rank_no_dupli_hand_less_t,count_hand_less_t)[1]
            
            if len(high_cards) != 0:
                #this list now contain the trio, next is to append the high cards
                quad = []
                t_x =[]
                for card in hand:
                    if card[1] == x:
                        quad += [card]
                comb = combinations(quad,3)
                combi = []
                for y in comb:
                    combi += [y]
                combi_convert = []
                for y in range(len(combi)):
                    combi_convert += [[]]
                for y in range(len(combi)):
                    combi_convert[y] += [combi[y][0]]
                    combi_convert[y] += [combi[y][1]]
                    combi_convert[y] += [combi[y][2]]
                        
                for y in high_cards:
                    for z in combi_convert:
                        t_x_with_h_c = z.copy()
                        t_x_with_h_c.extend(y)
                        trios += [t_x_with_h_c]
    
    return (len(trios),trios)

def trio_3(hand,rank,rank_no_dupli,count):
    '''
    input: hand is a list that contains 13 or 10 or 8 or 5 cards drawn from a standard deck without replacement
    with format (suit(4-diamond),rank(13-ace)).
        rank is a list that contains the rank of each card of a hand. it can have duplicated values.
        rank_no_dupli is a list that contains the ranks that are present in the hand. 
    it has no duplciated values and must be of descending order.
        count is a dictionary that contains the count of each rank in the hand.
        
    output: (a,b)
    a := len(b)
    b := a list that contains all the trio/s of length 3 that can be created from the hand. the list is sorted in descending order.
    '''
    ts = []
    for x in rank_no_dupli:
        if count[x] >= 3:
            ts += [x]
    
    if len(ts) == 0:
        return (0,[])
    
    #trios construction, which is the final product
    trios = []
    
    for x in ts:
        #first case
        if count[x] == 3:
            #this list is the trio, next is to append the high cards to it
            t_x = []
            for card in hand:
                if card[1] == x:
                    t_x += [card]
            trios += [t_x]
                    
        #second case
        elif count[x] == 4:
            quad = []
            for card in hand:
                if card[1] == x:
                    quad += [card]
            comb = combinations(quad,3)
            combi = []
            for y in comb:
                combi += [y]
            combi_convert = []
            for y in range(len(combi)):
                combi_convert += [[]]
            for y in range(len(combi)):
                combi_convert[y] += [combi[y][0]]
                combi_convert[y] += [combi[y][1]]
                combi_convert[y] += [combi[y][2]]
            for y in combi_convert:
                trios += [y]
    
    return (len(trios),trios)

def one_pair_5(hand,rank,rank_no_dupli,count):
    '''
    input: hand is a list that contains 13 or 10 or 8 or 5 cards drawn from a standard deck without replacement
    with format (suit(4-diamond),rank(13-ace)).
        rank is a list that contains the rank of each card of a hand. it can have duplicated values.
        rank_no_dupli is a list that contains the ranks that are present in the hand. 
    it has no duplciated values and must be of descending order.
        count is a dictionary that contains the count of each rank in the hand.
        
    output: (a,b)
    a := len(b)
    b := a list that contains all the one pair/s of length 5 that can be created from the hand. the list is sorted in descending order.
    '''    
    ops = []
    for x in rank_no_dupli:
        if count[x] >= 2:
            ops += [x]
    
    if len(ops) == 0:
        return (0,[])
    
    #else
    #one pairs construction, which is the final product
    onepairs = []
    
    for x in ops:
        #first case
        if count[x] == 2:
            hand_less_op = []
            for card in hand:
                if card[1] != x:
                    hand_less_op += [card]
            rank_hand_less_op = []
            for card in hand_less_op:
                rank_hand_less_op.append(card[1])
            rank_no_dupli_hand_less_op = list(set(rank_hand_less_op))
            rank_no_dupli_hand_less_op.sort(reverse=True)
            count_hand_less_op = {}
            for y in rank_no_dupli_hand_less_op:
                count_hand_less_op[y] = rank_hand_less_op.count(y)
            
            high_cards = high_card_3(hand_less_op,rank_hand_less_op,rank_no_dupli_hand_less_op,count_hand_less_op)[1]
            
            if len(high_cards) != 0:
                op_x = []
                for card in hand:
                    if card[1] == x:
                        op_x += [card]
                for y in high_cards:
                    op_x_with_h_c = op_x.copy()
                    op_x_with_h_c.extend(y)
                    onepairs += [op_x_with_h_c]

        #second case
        elif count[x] >= 3:
            hand_less_op = []
            for card in hand:
                if card[1] != x:
                    hand_less_op += [card]
            rank_hand_less_op = []
            for card in hand_less_op:
                rank_hand_less_op.append(card[1])
            rank_no_dupli_hand_less_op = list(set(rank_hand_less_op))
            rank_no_dupli_hand_less_op.sort(reverse=True)
            count_hand_less_op = {}
            for y in rank_no_dupli_hand_less_op:
                count_hand_less_op[y] = rank_hand_less_op.count(y)
            
            high_cards = high_card_3(hand_less_op,rank_hand_less_op,rank_no_dupli_hand_less_op,count_hand_less_op)[1]
            
            if len(high_cards) != 0:
                quad_or_trio = []
                op_x =[]
                for card in hand:
                    if card[1] == x:
                        quad_or_trio += [card]
                comb = combinations(quad_or_trio,2)
                combi = []
                for y in comb:
                    combi += [y]
                combi_convert = []
                for y in range(len(combi)):
                    combi_convert += [[]]
                for y in range(len(combi)):
                    combi_convert[y] += [combi[y][0]]
                    combi_convert[y] += [combi[y][1]]
                    
                for y in high_cards:
                    for z in combi_convert:
                        op_x_with_h_c = z.copy()
                        op_x_with_h_c.extend(y)
                        onepairs += [op_x_with_h_c]
    
    return (len(onepairs),onepairs)

def one_pair_3(hand,rank,rank_no_dupli,count):
    '''
    input: hand is a list that contains 13 or 3 cards drawn from a standard deck without replacement
    with format (suit(4-diamond),rank(13-ace)).
        rank is a list that contains the rank of each card of a hand. it can have duplicated values.
        rank_no_dupli is a list that contains the ranks that are present in the hand. 
    it has no duplciated values and must be of descending order.
        count is a dictionary that contains the count of each rank in the hand.
        
    output: (a,b)
    a := len(b)
    b := a list that contains all the one pair/s of length 5 that can be created from the hand. the list is sorted in descending order.
    '''    
    ops = []
    for x in rank_no_dupli:
        if count[x] >= 2:
            ops += [x]
    
    if len(ops) == 0:
        return (0,[])
    
    #else
    #one pairs construction, which is the final product
    onepairs = []
    
    for x in ops:
        #first case
        if count[x] == 2:
            hand_less_op = []
            for card in hand:
                if card[1] != x:
                    hand_less_op += [card]
            rank_hand_less_op = []
            for card in hand_less_op:
                rank_hand_less_op.append(card[1])
            rank_no_dupli_hand_less_op = list(set(rank_hand_less_op))
            rank_no_dupli_hand_less_op.sort(reverse=True)
            count_hand_less_op = {}
            for y in rank_no_dupli_hand_less_op:
                count_hand_less_op[y] = rank_hand_less_op.count(y)
            
            high_cards = high_card_1(hand_less_op,rank_hand_less_op,rank_no_dupli_hand_less_op,count_hand_less_op)[1]
            
            if len(high_cards) != 0:
                op_x = []
                for card in hand:
                    if card[1] == x:
                        op_x += [card]
                for y in high_cards:
                    op_x_with_h_c = op_x.copy()
                    op_x_with_h_c.extend(y)
                    onepairs += [op_x_with_h_c]

        #second case
        elif count[x] >= 3:
            hand_less_op = []
            for card in hand:
                if card[1] != x:
                    hand_less_op += [card]
            rank_hand_less_op = []
            for card in hand_less_op:
                rank_hand_less_op.append(card[1])
            rank_no_dupli_hand_less_op = list(set(rank_hand_less_op))
            rank_no_dupli_hand_less_op.sort(reverse=True)
            count_hand_less_op = {}
            for y in rank_no_dupli_hand_less_op:
                count_hand_less_op[y] = rank_hand_less_op.count(y)
            
            high_cards = high_card_1(hand_less_op,rank_hand_less_op,rank_no_dupli_hand_less_op,count_hand_less_op)[1]
            
            if len(high_cards) != 0:
                quad_or_trio = []
                op_x =[]
                for card in hand:
                    if card[1] == x:
                        quad_or_trio += [card]
                comb = combinations(quad_or_trio,2)
                combi = []
                for y in comb:
                    combi += [y]
                combi_convert = []
                for y in range(len(combi)):
                    combi_convert += [[]]
                for y in range(len(combi)):
                    combi_convert[y] += [combi[y][0]]
                    combi_convert[y] += [combi[y][1]]
                    
                for y in high_cards:
                    for z in combi_convert:
                        op_x_with_h_c = z.copy()
                        op_x_with_h_c.extend(y)
                        onepairs += [op_x_with_h_c]
    
    return (len(onepairs),onepairs)

def one_pair_2(hand,rank,rank_no_dupli,count):
    '''
    input: hand is a list that contains 13 or 10 or 8 or 5 cards drawn from a standard deck without replacement
    with format (suit(4-diamond),rank(13-ace)).
        rank is a list that contains the rank of each card of a hand. it can have duplicated values.
        rank_no_dupli is a list that contains the ranks that are present in the hand. 
    it has no duplciated values and must be of descending order.
        count is a dictionary that contains the count of each rank in the hand.
        
    output: (a,b)
    a := len(b)
    b := a list that contains all the one pair/s of length 2 that can be created from the hand. the list is sorted in descending order.
    '''
    ops = []
    for x in rank_no_dupli:
        if count[x] >= 2:
            ops += [x]
    
    if len(ops) == 0:
        return (0,[])
    
    #trios construction, which is the final product
    onepairs = []
    
    for x in ops:
        #first case
        if count[x] == 2:
            #this list is the trio, next is to append the high cards to it
            op_x = []
            for card in hand:
                if card[1] == x:
                    op_x += [card]
            onepairs += [op_x]
                    
        #second case
        elif count[x] >= 3:
            quad_trio = []
            for card in hand:
                if card[1] == x:
                    quad_trio += [card]
            comb = combinations(quad_trio,2)
            combi = []
            for y in comb:
                combi += [y]
            combi_convert = []
            for y in range(len(combi)):
                combi_convert += [[]]
            for y in range(len(combi)):
                combi_convert[y] += [combi[y][0]]
                combi_convert[y] += [combi[y][1]]
            for y in combi_convert:
                onepairs += [y]
    
    return (len(onepairs),onepairs)

def two_pairs_5(hand,rank,rank_no_dupli,count):
    '''
    input: hand is a list that contains 13 or 8 or 10 or 5 cards drawn from a standard deck without replacement
    with format (suit(4-diamond),rank(13-ace)).
        rank is a list that contains the rank of each card of a hand. it can have duplicated values.
        rank_no_dupli is a list that contains the ranks that are present in the hand. 
    it has no duplciated values and must be of descending order.
        count is a dictionary that contains the count of each rank in the hand.        
        
    output: (a,b)
    a := len(b)
    b := a list that contains all the two pair/s of length 5 that can be created from the hand. the list is sorted in descending order.
    '''
    ops = []
    for x in rank_no_dupli:
        if count[x] >= 2:
            ops += [x]
    
    if len(ops) <= 1:
        return (0,[])
    
    #else
    #two pairs construction, which is the final product
    twopairs = []
    
    combi = combinations_converted(ops,2)
    
    for x in combi:
        #gathering up the cards
        first_pair_cards = []
        for card in hand:
            if card[1] == x[0]:
                first_pair_cards += [card]
        second_pair_cards = []
        for card in hand:
            if card[1] == x[1]:
                second_pair_cards += [card]
                
        comb_fp = combinations_converted(first_pair_cards,2)
        comb_sp = combinations_converted(second_pair_cards,2)

        hand_less_tp = []
        for card in hand:
            if card[1] != x[0] and card[1] != x[1]:
                hand_less_tp += [card]
        rank_hand_less_tp = []
        for card in hand_less_tp:
            rank_hand_less_tp.append(card[1])
        rank_no_dupli_hand_less_tp = list(set(rank_hand_less_tp))
        rank_no_dupli_hand_less_tp.sort(reverse=True)
        count_hand_less_tp = {}
        for y in rank_no_dupli_hand_less_tp:
            count_hand_less_tp[y] = rank_hand_less_tp.count(y)

        high_cards = high_card_1(hand_less_tp,rank_hand_less_tp,rank_no_dupli_hand_less_tp,count_hand_less_tp)[1]
        
        twopair_without_hc = []
        for y in comb_fp:
            for z in comb_sp:
                t = y.copy()
                t.extend(z)
                twopair_without_hc += [t]
        
        two_pair = []
        for y in high_cards:
            for z in twopair_without_hc:
                tp = z.copy()
                tp.extend(y)
                two_pair += [tp]
                
        twopairs.extend(two_pair)
    
    return (len(twopairs),twopairs)

def full_house_5(hand,rank,rank_no_dupli,count):
    '''
    input: hand is a list that contains 13 or 8 or 10 or 5 cards drawn from a standard deck without replacement
    with format (suit(4-diamond),rank(13-ace)).
        rank is a list that contains the rank of each card of a hand. it can have duplicated values.
        rank_no_dupli is a list that contains the ranks that are present in the hand. 
    it has no duplciated values and must be of descending order.
        count is a dictionary that contains the count of each rank in the hand.        
        
    output: (a,b)
    a := len(b)
    b := a list that contains all the full house/s of length 5 that can be created from the hand. the list is sorted in descending order.
    '''
    ts = trio_3(hand,rank,rank_no_dupli,count)[1]
    ops = one_pair_2(hand,rank,rank_no_dupli,count)[1]
    
    if len(ts) == 0:
        return (0,[])
    
    #else
    #full houses construction, which is the final product
    fullhouses = []
    
    for x in ts:
        for y in ops:
            if x[-1][1] != y[-1][1]:
                fh = x.copy()
                fh.extend(y)
                fullhouses += [fh]
    
    if len(fullhouses) == 0:
        return (0,[])
    else:
        
        return (len(fullhouses),fullhouses)

def compare_5(h1, h2):
    '''returns the winning hand from two labelled hands.
    in the case of a draw, h1 is returned.'''
    if h1[0] > h2[0]:
        return h1
    elif h1[0] < h2[0]:
        return h2
    else: #same type
        if h1[0] == 10: #rf
            return h1
        elif h1[0] == 9:
            if h1[1][0][1] > h2[1][0][1]:
                return h1
            elif h1[1][0][1] < h2[1][0][1]:
                return h2
            else:
                return h1
        elif h1[0] == 8:
            if h1[1][0][1] > h2[1][0][1]:
                return h1
            elif h1[1][0][1] < h2[1][0][1]:
                return h2
            else:
                if h1[1][4][1] > h2[1][4][1]:
                    return h1
                elif h1[1][4][1] < h2[1][4][1]:
                    return h2
                else:
                    return h1
        elif h1[0] == 7:
            if h1[1][0][1] > h2[1][0][1]:
                return h1
            elif h1[1][0][1] < h2[1][0][1]:
                return h2
            else:
                if h1[1][3][1] > h2[1][3][1]:
                    return h1
                elif h1[1][3][1] < h2[1][3][1]:
                    return h2
                else:
                    return h1
        elif h1[0] == 6:
            if h1[1][0][1] > h2[1][0][1]:
                return h1
            elif h1[1][0][1] < h2[1][0][1]:
                return h2
            else:
                if h1[1][1][1] > h2[1][1][1]:
                    return h1
                elif h1[1][1][1] < h2[1][1][1]:
                    return h2
                else:
                    if h1[1][2][1] > h2[1][2][1]:
                        return h1
                    elif h1[1][2][1] < h2[1][2][1]:
                        return h2
                    else:
                        if h1[1][3][1] > h2[1][3][1]:
                            return h1
                        elif h1[1][3][1] < h2[1][3][1]:
                            return h2
                        else:
                            if h1[1][4][1] > h2[1][4][1]:
                                return h1
                            elif h1[1][4][1] < h2[1][4][1]:
                                return h2
                            else:
                                return h1
        elif h1[0] == 5:
            if h1[1][0][1] > h2[1][0][1]:
                return h1
            elif h1[1][0][1] < h2[1][0][1]:
                return h2
            else:
                return h1
        elif h1[0] == 4:
            if h1[1][0][1] > h2[1][0][1]:
                return h1
            elif h1[1][0][1] < h2[1][0][1]:
                return h2
            else:
                if h1[1][3][1] > h2[1][3][1]:
                    return h1
                elif h1[1][3][1] < h2[1][3][1]:
                    return h2
                else:
                    if h1[1][4][1] > h2[1][4][1]:
                        return h1
                    elif h1[1][4][1] < h2[1][4][1]:
                        return h2
                    else:
                        return h1
        elif h1[0] == 3:
            if h1[1][0][1] > h2[1][0][1]:
                return h1
            elif h1[1][0][1] < h2[1][0][1]:
                return h2
            else:
                if h1[1][2][1] > h2[1][2][1]:
                    return h1
                elif h1[1][2][1] < h2[1][2][1]:
                    return h2
                else:
                    if h1[1][4][1] > h2[1][4][1]:
                        return h1
                    elif h1[1][4][1] < h2[1][4][1]:
                        return h2
                    else:
                        return h1
        elif h1[0] == 2:
            if h1[1][0][1] > h2[1][0][1]:
                return h1
            elif h1[1][0][1] < h2[1][0][1]:
                return h2
            else:
                if h1[1][2][1] > h2[1][2][1]:
                    return h1
                elif h1[1][2][1] < h2[1][2][1]:
                    return h2
                else:
                    if h1[1][3][1] > h2[1][3][1]:
                        return h1
                    elif h1[1][3][1] < h2[1][3][1]:
                        return h2
                    else:
                        if h1[1][4][1] > h2[1][4][1]:
                            return h1
                        elif h1[1][4][1] < h2[1][4][1]:
                            return h2
                        else:
                            return h1
        else:
            if h1[1][0][1] > h2[1][0][1]:
                return h1
            elif h1[1][0][1] < h2[1][0][1]:
                return h2
            else:
                if h1[1][1][1] > h2[1][1][1]:
                    return h1
                elif h1[1][1][1] < h2[1][1][1]:
                    return h2
                else:
                    if h1[1][2][1] > h2[1][2][1]:
                        return h1
                    elif h1[1][2][1] < h2[1][2][1]:
                        return h2
                    else:
                        if h1[1][3][1] > h2[1][3][1]:
                            return h1
                        elif h1[1][3][1] < h2[1][3][1]:
                            return h2
                        else:
                            if h1[1][4][1] > h2[1][4][1]:
                                return h1
                            elif h1[1][4][1] < h2[1][4][1]:
                                return h2
                            else:
                                return h1
                            
def compare_5v3_or_3v3(h1, h2):
    '''h1:5, h2:3
    returns the winning hand from two labelled hands.
    in the case of a draw, h1 is returned.'''
    if h1[0] > h2[0]:
        return h1
    elif h1[0] < h2[0]:
        return h2
    else: #same type
        if h1[0] == 4: #trio
            if h1[1][0][1] > h2[1][0][1]:
                return h1
            else:
                return h2

        elif h1[0] == 2:
            if h1[1][0][1] > h2[1][0][1]:
                return h1
            elif h1[1][0][1] < h2[1][0][1]:
                return h2
            else:
                if h1[1][2][1] > h2[1][2][1]:
                    return h1
                elif h1[1][2][1] < h2[1][2][1]:
                    return h2
                else:
                    return h1
        else:
            if h1[1][0][1] > h2[1][0][1]:
                return h1
            elif h1[1][0][1] < h2[1][0][1]:
                return h2
            else:
                if h1[1][1][1] > h2[1][1][1]:
                    return h1
                elif h1[1][1][1] < h2[1][1][1]:
                    return h2
                else:
                    if h1[1][2][1] > h2[1][2][1]:
                        return h1
                    elif h1[1][2][1] < h2[1][2][1]:
                        return h2
                    else:
                        return h1
                    
def best_back_of_thirteen(hand):
    '''this can be used to label hand with length 5.'''
    rank = []
    for card in hand:
        rank.append(card[1])
    rank_no_dupli = list(set(rank))
    rank_no_dupli.sort(reverse=True)
    count = {}
    for x in rank_no_dupli:
        count[x] = rank.count(x)
    count[4] = rank.count(4)
    count[3] = rank.count(3)
    count[2] = rank.count(2)
    count[1] = rank.count(1)
    count[13] = rank.count(13)
    
    ten_5 = [(10,royal_flush_5), (9,straight_flush_5), (8,quad_5), (7,full_house_5), (6,flush_5), (5,straight_5), (4,trio_5), (3,two_pairs_5), (2,one_pair_5), (1,high_card_5)]

    for x in ten_5:
        a = x[1](hand,rank,rank_no_dupli,count)
        for y in a[1]:
            return (x[0],y)
        
def best_back_of_thirteen(hand):
    '''this can be used to label hand with length 5.'''
    rank = []
    for card in hand:
        rank.append(card[1])
    rank_no_dupli = list(set(rank))
    rank_no_dupli.sort(reverse=True)
    count = {}
    for x in rank_no_dupli:
        count[x] = rank.count(x)
    count[4] = rank.count(4)
    count[3] = rank.count(3)
    count[2] = rank.count(2)
    count[1] = rank.count(1)
    count[13] = rank.count(13)
    
    ten_5 = [(10,royal_flush_5), (9,straight_flush_5), (8,quad_5), (7,full_house_5), (6,flush_5), (5,straight_5), (4,trio_5), (3,two_pairs_5), (2,one_pair_5), (1,high_card_5)]

    for x in ten_5:
        a = x[1](hand,rank,rank_no_dupli,count)
        for y in a[1]:
            return (x[0],y)
        
def best_mid_of_thirteen(hand):
    rank = []
    for card in hand:
        rank.append(card[1])
    rank_no_dupli = list(set(rank))
    rank_no_dupli.sort(reverse=True)
    count = {}
    for x in rank_no_dupli:
        count[x] = rank.count(x)
    count[4] = rank.count(4)
    count[3] = rank.count(3)
    count[2] = rank.count(2)
    count[1] = rank.count(1)
    count[13] = rank.count(13)
    
    ten_5 = [(10,royal_flush_5), (9,straight_flush_5), (8,quad_5), (7,full_house_5), (6,flush_5), (5,straight_5), (4,trio_5), (3,two_pairs_5), (2,one_pair_5), (1,high_card_5)]
    
    best_mid = (0,'initial hand')
    
    for x in ten_5:
        temp = x[1](hand,rank,rank_no_dupli,count)
        if temp[0] != 0:
            for y in temp[1]:
                five = set(y)
                thirteen = set(hand)
                eight = list(thirteen.difference(five))
                best_mid = compare_5(best_mid,best_back_of_thirteen(eight))
        if best_mid[0] > x[0]-1:
            break

    return best_mid

def label_three(hand):
    '''this can be used to label hand with length 5.'''
    rank = []
    for card in hand:
        rank.append(card[1])
    rank_no_dupli = list(set(rank))
    rank_no_dupli.sort(reverse=True)
    count = {}
    for x in rank_no_dupli:
        count[x] = rank.count(x)
    
    ten_3 = [(4,trio_3), (2,one_pair_3), (1,high_card_3)]

    for x in ten_3:
        a = x[1](hand,rank,rank_no_dupli,count)
        for y in a[1]:
            return (x[0],y)

def best_front_of_thirteen(hand):
    rank = []
    for card in hand:
        rank.append(card[1])
    rank_no_dupli = list(set(rank))
    rank_no_dupli.sort(reverse=True)
    count = {}
    for x in rank_no_dupli:
        count[x] = rank.count(x)
    count[4] = rank.count(4)
    count[3] = rank.count(3)
    count[2] = rank.count(2)
    count[1] = rank.count(1)
    count[13] = rank.count(13)
    
    ten_5 = [(10,royal_flush_5), (9,straight_flush_5), (8,quad_5), (7,full_house_5), (6,flush_5), (5,straight_5), (4,trio_5), (3,two_pairs_5), (2,one_pair_5), (1,high_card_5)]
    ten_3 = [(4,trio_3), (2,one_pair_3), (1,high_card_3)]
    ten_5_dict = {10:royal_flush_5,9:straight_flush_5, 8:quad_5, 7:full_house_5, 6:flush_5, 5:straight_5, 4:trio_5, 3:two_pairs_5, 2:one_pair_5, 1:high_card_5}
    
    for x in ten_3:
        temp = x[1](hand,rank,rank_no_dupli,count)
        for y in temp[1]: #y is a three
            three = set(y)
            thirteen = set(hand)
            ten = list(thirteen.difference(three))

            rank_ten = []
            for card in ten:
                rank_ten.append(card[1])
            rank_no_dupli_ten = list(set(rank_ten))
            rank_no_dupli_ten.sort(reverse=True)
            count_ten = {}
            for z in rank_no_dupli:
                count_ten[z] = rank_ten.count(z)
            count_ten[4] = rank_ten.count(4)
            count_ten[3] = rank_ten.count(3)
            count_ten[2] = rank_ten.count(2)
            count_ten[1] = rank_ten.count(1)
            count_ten[13] = rank_ten.count(13)

            for z in range(x[0],11):
                five_of_ten = ten_5_dict[z](ten,rank_ten,rank_no_dupli_ten,count_ten)
                for a in five_of_ten[1]:
                    if compare_5v3_or_3v3((z,a),(x[0],y)) == (z,a):
                        back_five = set(a)
                        mid_five = list(set(ten).difference(back_five))
                        asdf = best_back_of_thirteen(mid_five)
                        if compare_5((z,a),asdf) == (z,a) and compare_5v3_or_3v3(asdf,(x[0],y)) == asdf:
                            return (x[0],y)
                        
def gen_attributes_of_hand(hand):
    '''returns (rank,rank_no_dupli,count)'''
    rank = []
    for card in hand:
        rank.append(card[1])
    rank_no_dupli = list(set(rank))
    rank_no_dupli.sort(reverse=True)
    count = {}
    for x in rank_no_dupli:
        count[x] = rank.count(x)
    count[4] = rank.count(4)
    count[3] = rank.count(3)
    count[2] = rank.count(2)
    count[1] = rank.count(1)
    count[13] = rank.count(13)
    
    return (rank,rank_no_dupli,count)

def converter_orig_to_coding(ranks):
	converted = []
	for x in ranks:
		if x == 1:
			converted += [13]
		elif x == 2:
			converted += [1]
		elif x == 3:
			converted += [2]
		elif x == 4:
			converted += [3]
		elif x == 5:
			converted += [4]
		elif x == 6:
			converted += [5]
		elif x == 7:
			converted += [6]
		elif x == 8:
			converted += [7]
		elif x == 9:
			converted += [8]
		elif x == 10:
			converted += [9]
		elif x == 11:
			converted += [10]
		elif x == 12:
			converted += [11]
		else:
			converted += [12]
	return converted
	
def converter_coding_to_orig(x):
		if x == 1:
			return 2
		elif x == 2:
			return 3
		elif x == 3:
			return 4
		elif x == 4:
			return 5
		elif x == 5:
			return 6
		elif x == 6:
			return 7
		elif x == 7:
			return 8
		elif x == 8:
			return 9
		elif x == 9:
			return 10
		elif x == 10:
			return 'J'
		elif x == 11:
			return 'Q'
		elif x == 12:
			return 'K'
		else:
			return 'A'
			
def hand_input_of_bmf(d,h,s,c):
	d = d.split()
	h = h.split()
	s = s.split()
	c = c.split()
	
	cards_string = [d,h,s,c]
	cards_ints = []
	for Suits in cards_string:
		ints = []
		for x in Suits:
			ints += [int(x)]
		cards_ints += [ints]
	
	d = converter_orig_to_coding(cards_ints[0])
	h = converter_orig_to_coding(cards_ints[1])
	s = converter_orig_to_coding(cards_ints[2])
	c = converter_orig_to_coding(cards_ints[3])
	cards = [d,h,s,c]
	
	hand = []
	counter = 4
	for Suit in cards:
		for card in Suit:
			hand += [(counter,card)]
		counter -= 1
	return hand
	

def int_to_names_funcs(integer):
	if integer == 1:
		return 'hc'
	elif integer == 2:
		return 'op'
	elif integer == 3:
		return 'tp'
	elif integer == 4:
		return 't'
	elif integer == 5:
		return 's'
	elif integer == 6:
		return 'f'
	elif integer == 7:
		return 'fh'
	elif integer == 8:
		return 'q'
	elif integer == 9:
		return 'sf'
	else:
		return 'rf'
		
def int_to_names_suits(integer):
    if integer == 4:
        return 'd'
    elif integer == 3:
        return 'h'
    elif integer == 2:
        return 's'
    else:
        return 'c'
    
def bmf(hand):
    '''returns a list that contains all labelled noncrack hands from a hand.'''
    noncrack_hands = []
    all_hands = []
    
    rank = []
    for card in hand:
        rank.append(card[1])
    rank_no_dupli = list(set(rank))
    rank_no_dupli.sort(reverse=True)
    count = {}
    for x in rank_no_dupli:
        count[x] = rank.count(x)
    count[4] = rank.count(4)
    count[3] = rank.count(3)
    count[2] = rank.count(2)
    count[1] = rank.count(1)
    count[13] = rank.count(13)
    
    ten_5 = [(10,royal_flush_5), (9,straight_flush_5), (8,quad_5), (7,full_house_5), (6,flush_5), (5,straight_5), (4,trio_5), (3,two_pairs_5), (2,one_pair_5), (1,high_card_5)]
    ten_3 = [(4,trio_3), (2,one_pair_3), (1,high_card_3)]
    ten_5_dict = {10:royal_flush_5,9:straight_flush_5, 8:quad_5, 7:full_house_5, 6:flush_5, 5:straight_5, 4:trio_5, 3:two_pairs_5, 2:one_pair_5, 1:high_card_5}

    five_of_thirteen = []
    for func in ten_5:
        temp = func[1](hand,rank,rank_no_dupli,count)
        for five in temp[1]:
            five_of_thirteen += [(func[0],five)]
            
    eights = []
    for five in five_of_thirteen:
        eight = set(hand).difference(set(five[1]))
        eights += [list(eight)]
    
    for x in range(len(eights)):
        gen_at = gen_attributes_of_hand(eights[x])
        
        five_of_eight = []
        for func in ten_5:
            temp = func[1](eights[x],gen_at[0],gen_at[1],gen_at[2])
            for five in temp[1]:
                five_of_eight += [(func[0],five)]
                
        three_of_three = []
        for five in five_of_eight:
            three = list(set(eights[x]).difference(set(five[1])))
            three_of_three += [label_three(three)]
        
        for y in range(len(five_of_eight)):
            all_hands += [(five_of_thirteen[x],five_of_eight[y],three_of_three[y])]
    
    for hands in all_hands:
        if compare_5(hands[0],hands[1]) == hands[0] and compare_5v3_or_3v3(hands[1],hands[2]) == hands[1]:
            noncrack_hands += [hands]
    
    noncrack_hands_less1and2 = []
    for hands in noncrack_hands:
        if hands[0][0] != 1 and hands[0][0] !=  2:
            noncrack_hands_less1and2 += [hands]
            
    #now we evaluate each element of the noncrack list using the bmf dataset.
    #to speedup this evaluation, create a dictionary that contains the necessary attributes of the dataset.
    memory = {}
    values = []
    for hands in noncrack_hands_less1and2:
        value = 'initial value'
        b = num_rows
        m = num_rows
        f = num_rows
        for y in range(bback_lesssuit_nodupli_labels.index(hands[0][0]),len(bback_lesssuit_nodupli)):
            if compare_5_lesssuit(hands[0],bback_lesssuit_nodupli[y]) == True:
                b = bback_count_cumulative[bback_lesssuit_nodupli[y]]
                break
        for y in range(bmid_lesssuit_nodupli_labels.index(hands[1][0]),len(bmid_lesssuit_nodupli)):
            if compare_5_lesssuit(hands[1],bmid_lesssuit_nodupli[y]) == True:
                m = bmid_count_cumulative[bmid_lesssuit_nodupli[y]]
                break
        for y in range(bfront_lesssuit_nodupli_labels.index(hands[2][0]),len(bfront_lesssuit_nodupli)):
            if compare_3_lesssuit(hands[2],bfront_lesssuit_nodupli[y]) == True:
                f = bfront_count_cumulative[bfront_lesssuit_nodupli[y]]
                break
        value = b+m+f
        memory[value] = hands
        values += [value]
    
    return memory[min(values)]
        
def bmf_output_converter(x):
	return ((int_to_names_funcs(x[0][0]),[(int_to_names_suits(x[0][1][0][0]),converter_coding_to_orig(x[0][1][0][1])),
                                    (int_to_names_suits(x[0][1][1][0]),converter_coding_to_orig(x[0][1][1][1])),
                                    (int_to_names_suits(x[0][1][2][0]),converter_coding_to_orig(x[0][1][2][1])),
                                    (int_to_names_suits(x[0][1][3][0]),converter_coding_to_orig(x[0][1][3][1])),
                                    (int_to_names_suits(x[0][1][4][0]),converter_coding_to_orig(x[0][1][4][1]))]),
            (int_to_names_funcs(x[1][0]),[(int_to_names_suits(x[1][1][0][0]),converter_coding_to_orig(x[1][1][0][1])),
                                    (int_to_names_suits(x[1][1][1][0]),converter_coding_to_orig(x[1][1][1][1])),
                                    (int_to_names_suits(x[1][1][2][0]),converter_coding_to_orig(x[1][1][2][1])),
                                    (int_to_names_suits(x[1][1][3][0]),converter_coding_to_orig(x[1][1][3][1])),
                                    (int_to_names_suits(x[1][1][4][0]),converter_coding_to_orig(x[1][1][4][1]))]),
            (int_to_names_funcs(x[2][0]),[(int_to_names_suits(x[2][1][0][0]),converter_coding_to_orig(x[2][1][0][1])),
                                    (int_to_names_suits(x[2][1][1][0]),converter_coding_to_orig(x[2][1][1][1])),
                                    (int_to_names_suits(x[2][1][2][0]),converter_coding_to_orig(x[2][1][2][1]))]))

def compare_5_lesssuit(h1,h2):
    '''h1 is the hand with suit, h2 is the hand less suit.
    return True if hand_withsuit beats or draws hand_lesssuit, else returns False.'''
    if h1[0] > h2[0]:
        return True
    elif h1[0] < h2[0]:
        return False
    else: #same type
        if h1[0] == 10: #rf
            return True
        elif h1[0] == 9:
            if h1[1][0][1] > h2[1][0]:
                return True
            elif h1[1][0][1] < h2[1][0]:
                return False
            else:
                return True
        elif h1[0] == 8:
            if h1[1][0][1] > h2[1][0]:
                return True
            elif h1[1][0][1] < h2[1][0]:
                return False
            else:
                if h1[1][4][1] > h2[1][4]:
                    return True
                elif h1[1][4][1] < h2[1][4]:
                    return False
                else:
                    return True
        elif h1[0] == 7:
            if h1[1][0][1] > h2[1][0]:
                return True
            elif h1[1][0][1] < h2[1][0]:
                return False
            else:
                if h1[1][3][1] > h2[1][3]:
                    return True
                elif h1[1][3][1] < h2[1][3]:
                    return False
                else:
                    return True
        elif h1[0] == 6:
            if h1[1][0][1] > h2[1][0]:
                return True
            elif h1[1][0][1] < h2[1][0]:
                return False
            else:
                if h1[1][1][1] > h2[1][1]:
                    return True
                elif h1[1][1][1] < h2[1][1]:
                    return False
                else:
                    if h1[1][2][1] > h2[1][2]:
                        return True
                    elif h1[1][2][1] < h2[1][2]:
                        return False
                    else:
                        if h1[1][3][1] > h2[1][3]:
                            return True
                        elif h1[1][3][1] < h2[1][3]:
                            return False
                        else:
                            if h1[1][4][1] > h2[1][4]:
                                return True
                            elif h1[1][4][1] < h2[1][4]:
                                return False
                            else:
                                return True
        elif h1[0] == 5:
            if h1[1][0][1] > h2[1][0]:
                return True
            elif h1[1][0][1] < h2[1][0]:
                return False
            else:
                return True
        elif h1[0] == 4:
            if h1[1][0][1] > h2[1][0]:
                return True
            elif h1[1][0][1] < h2[1][0]:
                return False
            else:
                if h1[1][3][1] > h2[1][3]:
                    return True
                elif h1[1][3][1] < h2[1][3]:
                    return False
                else:
                    if h1[1][4][1] > h2[1][4]:
                        return True
                    elif h1[1][4][1] < h2[1][4]:
                        return False
                    else:
                        return True
        elif h1[0] == 3:
            if h1[1][0][1] > h2[1][0]:
                return True
            elif h1[1][0][1] < h2[1][0]:
                return False
            else:
                if h1[1][2][1] > h2[1][2]:
                    return True
                elif h1[1][2][1] < h2[1][2]:
                    return False
                else:
                    if h1[1][4][1] > h2[1][4]:
                        return True
                    elif h1[1][4][1] < h2[1][4]:
                        return False
                    else:
                        return True
        elif h1[0] == 2:
            if h1[1][0][1] > h2[1][0]:
                return True
            elif h1[1][0][1] < h2[1][0]:
                return False
            else:
                if h1[1][2][1] > h2[1][2]:
                    return True
                elif h1[1][2][1] < h2[1][2]:
                    return False
                else:
                    if h1[1][3][1] > h2[1][3]:
                        return True
                    elif h1[1][3][1] < h2[1][3]:
                        return False
                    else:
                        if h1[1][4][1] > h2[1][4]:
                            return True
                        elif h1[1][4][1] < h2[1][4]:
                            return False
                        else:
                            return True
        else:
            if h1[1][0][1] > h2[1][0]:
                return True
            elif h1[1][0][1] < h2[1][0]:
                return False
            else:
                if h1[1][1][1] > h2[1][1]:
                    return True
                elif h1[1][1][1] < h2[1][1]:
                    return False
                else:
                    if h1[1][2][1] > h2[1][2]:
                        return True
                    elif h1[1][2][1] < h2[1][2]:
                        return False
                    else:
                        if h1[1][3][1] > h2[1][3]:
                            return True
                        elif h1[1][3][1] < h2[1][3]:
                            return False
                        else:
                            if h1[1][4][1] > h2[1][4]:
                                return True
                            elif h1[1][4][1] < h2[1][4]:
                                return False
                            else:
                                return True


def compare_3_lesssuit(h1,h2):
    if h1[0] > h2[0]:
        return True
    elif h1[0] < h2[0]:
        return False
    else: #same type
        if h1[0] == 4: #trio
            if h1[1][0][1] > h2[1][0]:
                return True
            else:
                return False

        elif h1[0] == 2:
            if h1[1][0][1] > h2[1][0]:
                return True
            elif h1[1][0][1] < h2[1][0]:
                return False
            else:
                if h1[1][2][1] > h2[1][2]:
                    return True
                elif h1[1][2][1] < h2[1][2]:
                    return False
                else:
                    return True
        else:
            if h1[1][0][1] > h2[1][0]:
                return True
            elif h1[1][0][1] < h2[1][0]:
                return False
            else:
                if h1[1][1][1] > h2[1][1]:
                    return True
                elif h1[1][1][1] < h2[1][1]:
                    return False
                else:
                    if h1[1][2][1] > h2[1][2]:
                        return True
                    elif h1[1][2][1] < h2[1][2]:
                        return False
                    else:
                        return True

In [4]:
#grabing the datasets from storage
#note that these are the sorted datasets
front_dataset = pd.read_csv(dataset_version + '_front.csv')
mid_dataset = pd.read_csv(dataset_version + '_mid.csv')
back_dataset = pd.read_csv(dataset_version + '_back.csv')

#converting them back to a more useful form
bback = []
bmid = []
bfront = []
for x in range(num_rows):
    bfront += [(front_dataset['bfront_label'][x],[(front_dataset['bfront_c1_suit'][x],front_dataset['bfront_c1_rank'][x]),(front_dataset['bfront_c2_suit'][x],front_dataset['bfront_c2_rank'][x]),(front_dataset['bfront_c3_suit'][x],front_dataset['bfront_c3_rank'][x])])]
    bmid += [(mid_dataset['bmid_label'][x],[(mid_dataset['bmid_c1_suit'][x],mid_dataset['bmid_c1_rank'][x]),(mid_dataset['bmid_c2_suit'][x],mid_dataset['bmid_c2_rank'][x]),(mid_dataset['bmid_c3_suit'][x],mid_dataset['bmid_c3_rank'][x]),(mid_dataset['bmid_c4_suit'][x],mid_dataset['bmid_c4_rank'][x]),(mid_dataset['bmid_c5_suit'][x],mid_dataset['bmid_c5_rank'][x])])]
    bback += [(back_dataset['bback_label'][x],[(back_dataset['bback_c1_suit'][x],back_dataset['bback_c1_rank'][x]),(back_dataset['bback_c2_suit'][x],back_dataset['bback_c2_rank'][x]),(back_dataset['bback_c3_suit'][x],back_dataset['bback_c3_rank'][x]),(back_dataset['bback_c4_suit'][x],back_dataset['bback_c4_rank'][x]),(back_dataset['bback_c5_suit'][x],back_dataset['bback_c5_rank'][x])])]

In [5]:
bmid.remove((2, [(1, 2), (4, 2), (1, 13), (2, 12), (4, 11)]))
bmid.append((1, [(1, 13), (4, 12), (1, 10), (2, 9), (4, 6)]))

In [6]:
bback_count = {}
bback_lesssuit = []
for x in bback:
    bback_lesssuit += [(x[0],(x[1][0][1],x[1][1][1],x[1][2][1],x[1][3][1],x[1][4][1]))]
bback_lesssuit_nodupli = list(set(bback_lesssuit))
bback_lesssuit_nodupli.sort(reverse=True)
for x in bback_lesssuit_nodupli:
    bback_count[x] = bback_lesssuit.count(x)
    
bmid_count = {}
bmid_lesssuit = []
for x in bmid:
    bmid_lesssuit += [(x[0],(x[1][0][1],x[1][1][1],x[1][2][1],x[1][3][1],x[1][4][1]))]
bmid_lesssuit_nodupli = list(set(bmid_lesssuit))
bmid_lesssuit_nodupli.sort(reverse=True)
for x in bmid_lesssuit_nodupli:
    bmid_count[x] = bmid_lesssuit.count(x)
    
bfront_count = {}
bfront_lesssuit = []
for x in bfront:
    bfront_lesssuit += [(x[0],(x[1][0][1],x[1][1][1],x[1][2][1]))]
bfront_lesssuit_nodupli = list(set(bfront_lesssuit))
bfront_lesssuit_nodupli.sort(reverse=True)
for x in bfront_lesssuit_nodupli:
    bfront_count[x] = bfront_lesssuit.count(x)
    
bback_count_cumulative = {}
for x in range(len(bback_lesssuit_nodupli)):
    c = 0
    for y in range(x+1):
        c += bback_count[bback_lesssuit_nodupli[y]]
    bback_count_cumulative[bback_lesssuit_nodupli[x]] = c
    
bmid_count_cumulative = {}
for x in range(len(bmid_lesssuit_nodupli)):
    c = 0
    for y in range(x+1):
        c += bmid_count[bmid_lesssuit_nodupli[y]]
    bmid_count_cumulative[bmid_lesssuit_nodupli[x]] = c
    
bfront_count_cumulative = {}
for x in range(len(bfront_lesssuit_nodupli)):
    c = 0
    for y in range(x+1):
        c += bfront_count[bfront_lesssuit_nodupli[y]]
    bfront_count_cumulative[bfront_lesssuit_nodupli[x]] = c
    
bback_lesssuit_nodupli_labels = []
for x in bback_lesssuit_nodupli:
    bback_lesssuit_nodupli_labels += [x[0]]
    
bmid_lesssuit_nodupli_labels = []
for x in bmid_lesssuit_nodupli:
    bmid_lesssuit_nodupli_labels += [x[0]]

bfront_lesssuit_nodupli_labels = []
for x in bfront_lesssuit_nodupli:
    bfront_lesssuit_nodupli_labels += [x[0]]

In [7]:
# #SORTED DATA STATISTICS
# bback_dist = {}
# bback_labels = []
# for x in bback:
# 	bback_labels += [x[0]]
# for x in range(10,0,-1):
# 	bback_dist[x] = bback_labels.count(x)
# print(bback_dist)

# bmid_dist = {}
# bmid_labels = []
# for x in bmid:
# 	bmid_labels += [x[0]]
# for x in range(10,0,-1):
# 	bmid_dist[x] = bmid_labels.count(x)
# print(bmid_dist)

# bfront_dist = {}
# bfront_labels = []
# for x in bfront:
# 	bfront_labels += [x[0]]
# for x in range(10,0,-1):
# 	bfront_dist[x] = bfront_labels.count(x)
# print(bfront_dist)
# print('######################')
# set1 = set()
# for x in bback:
# 	set1.add((x[0],(x[1][0][1],x[1][1][1],x[1][2][1],x[1][3][1],x[1][4][1])))
# print(len(set1))

# set2 = set()
# for x in bmid:
# 	set2.add((x[0],(x[1][0][1],x[1][1][1],x[1][2][1],x[1][3][1],x[1][4][1])))
# print(len(set2))

# set3 = set()
# for x in bfront:
# 	set3.add((x[0],(x[1][0][1],x[1][1][1],x[1][2][1])))
# print(len(set3))

In [8]:
def play(entry):
    entry = entry.split(sep='+')
    
    diamond = entry[0]
    heart = entry[1]
    spade = entry[2]
    club = entry[3]
    
    a = bmf_output_converter(bmf(hand_input_of_bmf(diamond,heart,spade,club)))
    b = random.randint(0,1)
    c = 'initial value'
    if b == 0:
        c = 1
    else:
        c = 0

    print(a[2][0],a[2][1][0],a[2][1][1],a[2][1][2])
    print(b,a[1][0],a[1][1][0],a[1][1][1],a[1][1][2],a[1][1][3],a[1][1][4])
    print(c,a[0][0],a[0][1][0],a[0][1][1],a[0][1][2],a[0][1][3],a[0][1][4])

In [ ]:
entry = ''
play(entry)

In [ ]:
entry = ''
play(entry)